In [20]:
import coba as cb
import torch
from pathlib import Path
from collections import defaultdict, Counter
from itertools import islice, chain, count, product, repeat
from contextlib import nullcontext

data_dir = "../data"

import coba as cb

import warnings

import time
import csv
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator
from sklearn.preprocessing import RobustScaler, QuantileTransformer, MinMaxScaler, StandardScaler, Binarizer
from sklearn.feature_selection import  mutual_info_classif, f_classif, GenericUnivariateSelect
from sklearn.model_selection import cross_validate, cross_val_predict, StratifiedKFold, GridSearchCV, LeaveOneGroupOut, StratifiedShuffleSplit
from sklearn.metrics import balanced_accuracy_score, mean_squared_error, mean_absolute_error
from sklearn.pipeline import Pipeline

import torch
from parameterfree import COCOB

from sklearn.dummy import DummyRegressor, DummyClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from xgboost import XGBClassifier

from concurrent.futures import ProcessPoolExecutor

import torch
import torch.utils
import torch.utils.data
import coba as cb
from typing import Tuple, Optional

try:
    torch.set_num_threads(3)
    torch.set_num_interop_threads(3)
except RuntimeError:
    pass

c0 = "#444"
c1 = "#0072B2"
c2 = "#E69F00"
c3 = "#009E73"
c4 = "#56B4E9"
c5 = "#D55E00"
c6 = "#F0E442"
c7 = "#CC79A7"
c8 = "#000000"
c9 = "#332288"

torch.set_default_device('cpu')
plt.rc('font', **{'size': 20})

df = pd.read_csv(f"{data_dir}/all_features_1h_v3.csv")

G  = df["id_participant"].to_numpy()
X1 = df[[c for c in df.columns if c.startswith("acc_")]].to_numpy()
X2 = df[[c for c in df.columns if c.startswith("acc_") or c.startswith("gps_") or c.startswith("motion_")]].to_numpy()
Y1 =  df["ER_desire"].astype(float).to_numpy()
Y2 = (df["INT_availability"] == "yes").astype(float).to_numpy()

no_na = ~(np.isnan(Y1) | np.isnan(Y2))
#no_na = ~(np.isnan(Y2))

G  = G [no_na]
X1 = X1[no_na]
X2 = X2[no_na]
X3 = X2.copy()
X4 = X2.copy()
Y1 = Y1[no_na]
Y2 = Y2[no_na]

for g in set(G):
    Y1[G == g] = Binarizer(threshold=np.median(Y1[G == g])).fit_transform(np.expand_dims(Y1[G == g],axis=1)).squeeze()
    X1[G == g] = StandardScaler().fit_transform(X1[G == g])
    X2[G == g] = StandardScaler().fit_transform(X2[G == g])
    X3[G == g] = StandardScaler().fit_transform(X3[G == g])
    X4[G == g] = StandardScaler().fit_transform(X4[G == g])

X3 = np.concatenate([X3, np.expand_dims((df["Platform"] == "Android").astype(float).to_numpy(),1)[no_na]],axis=1)
X3 = np.concatenate([X3, np.expand_dims((df["Platform"] == "iPhone" ).astype(float).to_numpy(),1)[no_na]],axis=1)

X4 = np.concatenate([X4, np.expand_dims((df["Platform"] == "Android").astype(float).to_numpy(),1)[no_na]],axis=1)
X4 = np.concatenate([X4, np.expand_dims((df["Platform"] == "iPhone" ).astype(float).to_numpy(),1)[no_na]],axis=1)
X4 = np.concatenate([X4, np.expand_dims((df["tag"] == "evening" ).astype(float).to_numpy(),1)[no_na]],axis=1)
X4 = np.concatenate([X4, np.expand_dims((df["tag"] == "morning" ).astype(float).to_numpy(),1)[no_na]],axis=1)
X4 = np.concatenate([X4, np.expand_dims((df["tag"] == "afternoon" ).astype(float).to_numpy(),1)[no_na]],axis=1)

/home/mrucker/miniconda3/envs/diss/lib/python3.10/site-packages/sklearn/utils/extmath.py:1137: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/home/mrucker/miniconda3/envs/diss/lib/python3.10/site-packages/sklearn/utils/extmath.py:1142: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/home/mrucker/miniconda3/envs/diss/lib/python3.10/site-packages/sklearn/utils/extmath.py:1162: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count
/home/mrucker/miniconda3/envs/diss/lib/python3.10/site-packages/sklearn/utils/extmath.py:1137: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/home/mrucker/miniconda3/envs/diss/lib/python3.10/site-packages/sklearn/utils/extmath.py:1142: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/home/mrucker

In [ ]:
import coba as cb

class FeedForward(torch.nn.Sequential):
    """A Generic implementation of Feedforward Neural Network"""

    class SkipModule(torch.nn.Module):
        def __init__(self, layers):
            super().__init__()
            self.layers = layers
        def forward(self,X):
            return X + self.layers(X)

    def make_layer(self,curr_dim,spec):
        if isinstance(spec,float):
            return torch.nn.Dropout(spec), curr_dim
        if curr_dim is None and isinstance(spec,int):
            return None, spec
        if isinstance(spec,int):
            return torch.nn.Linear(curr_dim,spec),spec
        if spec == 'r':
            return torch.nn.ReLU(),curr_dim
        if spec == 'l':
            return torch.nn.LayerNorm(curr_dim),curr_dim
        if spec == 'b':
            return torch.nn.BatchNorm1d(curr_dim), curr_dim
        if spec == 's':
            return torch.nn.Sigmoid(),curr_dim
        if isinstance(spec,list):                
            return FeedForward.SkipModule(FeedForward([curr_dim] + spec)), curr_dim
        raise Exception("Bad Layer")

    def __init__(self, specs, rng=1):
        """Instantiate a Feedfoward network according to specifications.

        Args:
            specs: A sequence of layer specifications as follows:
                -1 -- replaced with the input feature width
                <int> -- a LinearLayer with output width equal to <int>
                [0,1] -- a Dropout layer with the given probability
                'l' -- a LayerNorm
                'b' -- a BatchNorm1d
                'r' -- a ReLU layer
                's' -- a Sigmoid layer
                [] -- a skip layer with the given specifications
        """

        torch.manual_seed(rng)
        layers,width = [],None
        for spec in specs:
            layer,width = self.make_layer(width,spec)
            if layer: layers.append(layer)
        super().__init__(*(layers or [torch.nn.Identity()]))
        self.params = {"specs": specs, "rng": rng }

class MyEnvironment:
    def __init__(self, train_X, train_Y, test_X, test_Y, g, rng):
        self.params = {'pid': g, 'rng': rng}
        self.train_X = train_X
        self.train_Y = train_Y.float()
        self.test_X = test_X
        self.test_Y = test_Y.float()

    def ssl(self,neg,sr):

        by_class  = {y: self.train_X[(self.train_Y==y)].tolist() for y in set(self.train_Y.tolist())}
        rng = cb.CobaRandom(self.params['rng'])

        def choose_unique(items,item):
            while items[i:=rng.randint(0,len(items)-1)]==item:
                pass
            return items[i]

        def choose_n(items,n):
            indexes = set()
            while True:
                indexes.add(rng.randint(0,len(items)-1))
                if len(indexes)==n: return [items[i] for i in indexes]

        for _ in range(sr):
            anchor, positive, negative = [], [], []
            for x,y in zip(self.train_X.tolist(),self.train_Y.tolist()):
                anchor.append(x)
                positive.append(choose_unique(by_class[y],x))
                negative.append(choose_n(by_class[1-y],neg))

            yield torch.tensor(anchor).float(), torch.tensor(positive).float(), torch.tensor(negative).float()

    def train(self):
        return self.train_X, self.train_Y

    def test(self):
        rng_indexes = cb.CobaRandom(self.params['rng']).shuffle(range(len(self.test_X)))
        return self.test_X[rng_indexes,:], self.test_Y[rng_indexes]

class MyEvaluator:
    def __init__(self, s1, ws):
        self.s1 = s1
        self.ws = ws
        self.params = {'s1': s1, 'ws':ws }

    def evaluate(self, env, lrn):        
        torch.set_num_threads(1)
        torch.manual_seed(1)
        
        X, Y = env.train()

        loss = torch.nn.L1Loss(reduction='none')
        torch_loader  = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X),batch_size=16,shuffle=True)

        f = FeedForward(self.s1)
        fopt = COCOB(f.parameters())
        f.train()

        for _ in range(self.ws):
            for (_X,) in torch_loader:
                fopt.zero_grad()
                loss(f(torch.nan_to_num(_X)),_X).nanmean().backward()
                fopt.step()

        f.eval()
        X, Y = env.test()
        for i in range(35):
            with torch.no_grad():
                yield {'l1':loss(f(torch.nan_to_num(X)),X).nanmean()}

def make_envs(X, Y, G):
    X, Y, G = torch.tensor(X).float(), torch.tensor(Y).float(), torch.tensor(G)
    all_equal = set()
    envs = []
    for rng in range(1):
        for g in sorted(set(G.tolist())):
            if (g==G).sum() < 50: continue
            trn_x,trn_y = X[g!=G],Y[g!=G]
            tst_x,tst_y = X[g==G],Y[g==G]

            if len(set(tst_y.tolist())) == 1:
                all_equal.add(g)
            else:
                envs.append(MyEnvironment(trn_x,trn_y,tst_x,tst_y,g,rng))
    if all_equal: print(f"All Equal, no environment added for {sorted(all_equal)}") 
    return envs

lrns = [ FeedForward(specs=[], rng=1) ]

vals = lambda x: [
    MyEvaluator((len(x),30,0.5,'r',len(x)),3),
    MyEvaluator((len(x),30,'l','r',len(x)),3),
    MyEvaluator((len(x),45,'l','r',len(x)),3),
]

envs = make_envs(X3,Y2,G)
cb.Experiment(envs,lrns,vals(X3[0])).run('../logs/outae1.log',processes=35,quiet=True)


All Equal, no environment added for [2, 25, 61, 82, 87, 89, 99, 103, 111, 117, 146, 196, 203, 227, 246, 311, 319, 357, 377, 390, 451, 455]
2025-05-10 17:46:42 -- pid-633466 -- Unexpected exception:

  File "/home/mrucker/Projects/coba/coba/experiments/core.py", line 197, in run
    Pipes.join(workitems, chunker, process, preamble, encode, sink).run()
  File "/home/mrucker/Projects/coba/coba/pipes/lines.py", line 163, in run
    sink.write(item)
  File "/home/mrucker/Projects/coba/coba/pipes/sinks.py", line 98, in write
    batch = self._get_batch(lines)
  File "/home/mrucker/Projects/coba/coba/pipes/sinks.py", line 106, in _get_batch
    if self._batch: batch = list(batch)
  File "/home/mrucker/Projects/coba/coba/results/core.py", line 517, in filter
    for item in transactions:
  File "/home/mrucker/Projects/coba/coba/multiprocessing.py", line 72, in filter
    yield from Multiprocessor(filter, self._processes, self._maxtasksperchild).filter(items)
  File "/home/mrucker/Projects/coba

Background process 694428 failed unexpectedly with exit code 1.
Background process 694460 failed unexpectedly with exit code 1.
Background process 694490 failed unexpectedly with exit code 1.
Background process 694474 failed unexpectedly with exit code 1.
Background process 694446 failed unexpectedly with exit code 1.
Background process 694458 failed unexpectedly with exit code 1.
Background process 694488 failed unexpectedly with exit code 1.
Background process 694476 failed unexpectedly with exit code 1.
Background process 694462 failed unexpectedly with exit code 1.
Background process 694456 failed unexpectedly with exit code 1.
Background process 694486 failed unexpectedly with exit code 1.
Background process 694480 failed unexpectedly with exit code 1.
Background process 694464 failed unexpectedly with exit code 1.
Background process 694470 failed unexpectedly with exit code 1.
Background process 694454 failed unexpectedly with exit code 1.
Background process 694472 failed unexpec

In [ ]:
import coba as cb

class FeedForward(torch.nn.Sequential):
    """A Generic implementation of Feedforward Neural Network"""

    class SkipModule(torch.nn.Module):
        def __init__(self, layers):
            super().__init__()
            self.layers = layers
        def forward(self,X):
            return X + self.layers(X)

    def make_layer(self,curr_dim,spec):
        if isinstance(spec,float):
            return torch.nn.Dropout(spec), curr_dim
        if curr_dim is None and isinstance(spec,int):
            return None, spec
        if isinstance(spec,int):
            return torch.nn.Linear(curr_dim,spec),spec
        if spec == 'r':
            return torch.nn.ReLU(),curr_dim
        if spec == 'l':
            return torch.nn.LayerNorm(curr_dim),curr_dim
        if spec == 'b':
            return torch.nn.BatchNorm1d(curr_dim), curr_dim
        if spec == 's':
            return torch.nn.Sigmoid(),curr_dim
        if isinstance(spec,list):                
            return FeedForward.SkipModule(FeedForward([curr_dim] + spec)), curr_dim
        raise Exception("Bad Layer")

    def __init__(self, specs, rng=1):
        """Instantiate a Feedfoward network according to specifications.

        Args:
            specs: A sequence of layer specifications as follows:
                -1 -- replaced with the input feature width
                <int> -- a LinearLayer with output width equal to <int>
                [0,1] -- a Dropout layer with the given probability
                'l' -- a LayerNorm
                'b' -- a BatchNorm1d
                'r' -- a ReLU layer
                's' -- a Sigmoid layer
                [] -- a skip layer with the given specifications
        """

        torch.manual_seed(rng)
        layers,width = [],None
        for spec in specs:
            layer,width = self.make_layer(width,spec)
            if layer: layers.append(layer)
        super().__init__(*(layers or [torch.nn.Identity()]))
        self.params = {"specs": specs, "rng": rng }

class MyEnvironment:
    def __init__(self, train_X, train_Y, test_X, test_Y, g, rng):
        self.params = {'pid': g, 'rng': rng}
        self.train_X = train_X
        self.train_Y = train_Y.float()
        self.test_X = test_X
        self.test_Y = test_Y.float()

    def ssl(self,neg,sr):

        by_class  = {y: self.train_X[(self.train_Y==y)].tolist() for y in set(self.train_Y.tolist())}
        rng = cb.CobaRandom(self.params['rng'])

        def choose_unique(items,item):
            while items[i:=rng.randint(0,len(items)-1)]==item:
                pass
            return items[i]

        def choose_n(items,n):
            indexes = set()
            while True:
                indexes.add(rng.randint(0,len(items)-1))
                if len(indexes)==n: return [items[i] for i in indexes]

        for _ in range(sr):
            anchor, positive, negative = [], [], []
            for x,y in zip(self.train_X.tolist(),self.train_Y.tolist()):
                anchor.append(x)
                positive.append(choose_unique(by_class[y],x))
                negative.append(choose_n(by_class[1-y],neg))

            yield torch.tensor(anchor).float(), torch.tensor(positive).float(), torch.tensor(negative).float()

    def train(self):
        return self.train_X, self.train_Y

    def test(self):
        rng_indexes = cb.CobaRandom(self.params['rng']).shuffle(range(len(self.test_X)))
        return self.test_X[rng_indexes,:], self.test_Y[rng_indexes]

class MyEvaluator:
    def __init__(self, s1, ws):
        self.s1 = s1
        self.ws = ws
        self.params = {'s1': s1, 'ws':ws }

    def evaluate(self, env, lrn):        
        torch.set_num_threads(1)
        torch.manual_seed(1)
        
        X, Y = env.train()

        loss = torch.nn.L1Loss(reduction='none')
        torch_loader  = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X),batch_size=16,shuffle=True)

        f = FeedForward(self.s1)
        fopt = COCOB(f.parameters())
        f.train()

        for _ in range(self.ws):
            for (_X,) in torch_loader:
                fopt.zero_grad()
                loss(f(torch.nan_to_num(_X)),_X).nanmean().backward()
                fopt.step()

        f.eval()
        X, Y = env.test()
        for i in range(35):
            with torch.no_grad():
                yield {'l1':loss(f(torch.nan_to_num(X)),X).nanmean()}

def make_envs(X, Y, G):
    X, Y, G = torch.tensor(X).float(), torch.tensor(Y).float(), torch.tensor(G)
    all_equal = set()
    envs = []
    for rng in range(1):
        for g in sorted(set(G.tolist())):
            if (g==G).sum() < 50: continue
            trn_x,trn_y = X[g!=G],Y[g!=G]
            tst_x,tst_y = X[g==G],Y[g==G]

            if len(set(tst_y.tolist())) == 1:
                all_equal.add(g)
            else:
                envs.append(MyEnvironment(trn_x,trn_y,tst_x,tst_y,g,rng))
    if all_equal: print(f"All Equal, no environment added for {sorted(all_equal)}") 
    return envs

lrns = [ FeedForward(specs=[], rng=1) ]

vals = lambda x: [
    MyEvaluator((len(x),30,0.5,'r',len(x)),3),
    MyEvaluator((len(x),30,0.5,'r',len(x)),4),
    MyEvaluator((len(x),30,0.5,'r',len(x)),5),
    MyEvaluator((len(x),30,'l','r',len(x)),3),
    MyEvaluator((len(x),30,'l','r',len(x)),4),
    MyEvaluator((len(x),30,'l','r',len(x)),5),
]

envs = make_envs(X3,Y2,G)
cb.Experiment(envs,lrns,vals(X3[0])).run('../logs/outae2.log',processes=35,quiet=True)


All Equal, no environment added for [2, 25, 61, 82, 87, 89, 99, 103, 111, 117, 146, 196, 203, 227, 246, 311, 319, 357, 377, 390, 451, 455]
{'Learners': 1, 'Environments': 190, 'Interactions': 38500}


In [ ]:
import coba as cb

class FeedForward(torch.nn.Sequential):
    """A Generic implementation of Feedforward Neural Network"""

    class SkipModule(torch.nn.Module):
        def __init__(self, layers):
            super().__init__()
            self.layers = layers
        def forward(self,X):
            return X + self.layers(X)

    def make_layer(self,curr_dim,spec):
        if isinstance(spec,float):
            return torch.nn.Dropout(spec), curr_dim
        if curr_dim is None and isinstance(spec,int):
            return None, spec
        if isinstance(spec,int):
            return torch.nn.Linear(curr_dim,spec),spec
        if spec == 'r':
            return torch.nn.ReLU(),curr_dim
        if spec == 'l':
            return torch.nn.LayerNorm(curr_dim),curr_dim
        if spec == 'b':
            return torch.nn.BatchNorm1d(curr_dim), curr_dim
        if spec == 's':
            return torch.nn.Sigmoid(),curr_dim
        if isinstance(spec,list):                
            return FeedForward.SkipModule(FeedForward([curr_dim] + spec)), curr_dim
        raise Exception("Bad Layer")

    def __init__(self, specs, rng=1):
        """Instantiate a Feedfoward network according to specifications.

        Args:
            specs: A sequence of layer specifications as follows:
                -1 -- replaced with the input feature width
                <int> -- a LinearLayer with output width equal to <int>
                [0,1] -- a Dropout layer with the given probability
                'l' -- a LayerNorm
                'b' -- a BatchNorm1d
                'r' -- a ReLU layer
                's' -- a Sigmoid layer
                [] -- a skip layer with the given specifications
        """

        torch.manual_seed(rng)
        layers,width = [],None
        for spec in specs:
            layer,width = self.make_layer(width,spec)
            if layer: layers.append(layer)
        super().__init__(*(layers or [torch.nn.Identity()]))
        self.params = {"specs": specs, "rng": rng }

class MyEnvironment:
    def __init__(self, train_X, train_Y, test_X, test_Y, g, rng):
        self.params = {'pid': g, 'rng': rng}
        self.train_X = train_X
        self.train_Y = train_Y.float()
        self.test_X = test_X
        self.test_Y = test_Y.float()

    def ssl(self,neg,sr):

        by_class  = {y: self.train_X[(self.train_Y==y)].tolist() for y in set(self.train_Y.tolist())}
        rng = cb.CobaRandom(self.params['rng'])

        def choose_unique(items,item):
            while items[i:=rng.randint(0,len(items)-1)]==item:
                pass
            return items[i]

        def choose_n(items,n):
            indexes = set()
            while True:
                indexes.add(rng.randint(0,len(items)-1))
                if len(indexes)==n: return [items[i] for i in indexes]

        for _ in range(sr):
            anchor, positive, negative = [], [], []
            for x,y in zip(self.train_X.tolist(),self.train_Y.tolist()):
                anchor.append(x)
                positive.append(choose_unique(by_class[y],x))
                negative.append(choose_n(by_class[1-y],neg))

            yield torch.tensor(anchor).float(), torch.tensor(positive).float(), torch.tensor(negative).float()

    def train(self):
        return self.train_X, self.train_Y

    def test(self):
        rng_indexes = cb.CobaRandom(self.params['rng']).shuffle(range(len(self.test_X)))
        return self.test_X[rng_indexes,:], self.test_Y[rng_indexes]

class MyEvaluator:
    def __init__(self, s1, ws):
        self.s1 = s1
        self.ws = ws
        self.params = {'s1': s1, 'ws':ws }

    def evaluate(self, env, lrn):        
        torch.set_num_threads(1)
        torch.manual_seed(1)
        
        X, Y = env.train()

        loss = torch.nn.L1Loss(reduction='none')
        torch_loader  = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X),batch_size=16,shuffle=True)

        f = FeedForward(self.s1)
        fopt = COCOB(f.parameters())
        f.train()

        for _ in range(self.ws):
            for (_X,) in torch_loader:
                fopt.zero_grad()
                loss(f(torch.nan_to_num(_X)),_X).nanmean().backward()
                fopt.step()

        f.eval()
        X, Y = env.test()
        for i in range(35):
            with torch.no_grad():
                yield {'l1':loss(f(torch.nan_to_num(X)),X).nanmean()}

def make_envs(X, Y, G):
    X, Y, G = torch.tensor(X).float(), torch.tensor(Y).float(), torch.tensor(G)
    all_equal = set()
    envs = []
    for rng in range(1):
        for g in sorted(set(G.tolist())):
            if (g==G).sum() < 50: continue
            trn_x,trn_y = X[g!=G],Y[g!=G]
            tst_x,tst_y = X[g==G],Y[g==G]

            if len(set(tst_y.tolist())) == 1:
                all_equal.add(g)
            else:
                envs.append(MyEnvironment(trn_x,trn_y,tst_x,tst_y,g,rng))
    if all_equal: print(f"All Equal, no environment added for {sorted(all_equal)}") 
    return envs

lrns = [ FeedForward(specs=[], rng=1) ]

vals = lambda x: [
    MyEvaluator((len(x),30,0.2,'r',len(x)),6),
    MyEvaluator((len(x),30,'l','r',len(x)),4),
    MyEvaluator((len(x),20,'l','r',20,'l','r',len(x)),4),
]

envs = make_envs(X3,Y2,G)
cb.Experiment(envs,lrns,vals(X3[0])).run('../logs/outae3.log',processes=35,quiet=True)


In [ ]:
import coba as cb

class FeedForward(torch.nn.Sequential):
    """A Generic implementation of Feedforward Neural Network"""

    class SkipModule(torch.nn.Module):
        def __init__(self, layers):
            super().__init__()
            self.layers = layers
        def forward(self,X):
            return X + self.layers(X)

    def make_layer(self,curr_dim,spec):
        if isinstance(spec,float):
            return torch.nn.Dropout(spec), curr_dim
        if curr_dim is None and isinstance(spec,int):
            return None, spec
        if isinstance(spec,int):
            return torch.nn.Linear(curr_dim,spec),spec
        if spec == 'r':
            return torch.nn.ReLU(),curr_dim
        if spec == 'l':
            return torch.nn.LayerNorm(curr_dim),curr_dim
        if spec == 'b':
            return torch.nn.BatchNorm1d(curr_dim), curr_dim
        if spec == 's':
            return torch.nn.Sigmoid(),curr_dim
        if isinstance(spec,list):                
            return FeedForward.SkipModule(FeedForward([curr_dim] + spec)), curr_dim
        raise Exception("Bad Layer")

    def __init__(self, specs, rng=1):
        """Instantiate a Feedfoward network according to specifications.

        Args:
            specs: A sequence of layer specifications as follows:
                -1 -- replaced with the input feature width
                <int> -- a LinearLayer with output width equal to <int>
                [0,1] -- a Dropout layer with the given probability
                'l' -- a LayerNorm
                'b' -- a BatchNorm1d
                'r' -- a ReLU layer
                's' -- a Sigmoid layer
                [] -- a skip layer with the given specifications
        """

        torch.manual_seed(rng)
        layers,width = [],None
        for spec in specs:
            layer,width = self.make_layer(width,spec)
            if layer: layers.append(layer)
        super().__init__(*(layers or [torch.nn.Identity()]))
        self.params = {"specs": specs, "rng": rng }

class MyEnvironment:
    def __init__(self, train_X, train_Y, test_X, test_Y, g, rng):
        self.params = {'pid': g, 'rng': rng}
        self.train_X = train_X
        self.train_Y = train_Y.float()
        self.test_X = test_X
        self.test_Y = test_Y.float()

    def ssl(self,neg,sr):

        by_class  = {y: self.train_X[(self.train_Y==y)].tolist() for y in set(self.train_Y.tolist())}
        rng = cb.CobaRandom(self.params['rng'])

        def choose_unique(items,item):
            while items[i:=rng.randint(0,len(items)-1)]==item:
                pass
            return items[i]

        def choose_n(items,n):
            indexes = set()
            while True:
                indexes.add(rng.randint(0,len(items)-1))
                if len(indexes)==n: return [items[i] for i in indexes]

        for _ in range(sr):
            anchor, positive, negative = [], [], []
            for x,y in zip(self.train_X.tolist(),self.train_Y.tolist()):
                anchor.append(x)
                positive.append(choose_unique(by_class[y],x))
                negative.append(choose_n(by_class[1-y],neg))

            yield torch.tensor(anchor).float(), torch.tensor(positive).float(), torch.tensor(negative).float()

    def train(self):
        return self.train_X, self.train_Y

    def test(self):
        rng_indexes = cb.CobaRandom(self.params['rng']).shuffle(range(len(self.test_X)))
        return self.test_X[rng_indexes,:], self.test_Y[rng_indexes]

class MyEvaluator:
    def __init__(self, s1, ws):
        self.s1 = s1
        self.ws = ws
        self.params = {'s1': s1, 'ws':ws }

    def evaluate(self, env, lrn):        
        torch.set_num_threads(1)
        torch.manual_seed(1)
        
        X, Y = env.train()

        loss = torch.nn.L1Loss(reduction='none')
        torch_loader  = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X),batch_size=16,shuffle=True)

        f = FeedForward(self.s1)
        fopt = COCOB(f.parameters())
        f.train()

        for _ in range(self.ws):
            for (_X,) in torch_loader:
                fopt.zero_grad()
                loss(f(torch.nan_to_num(_X)),_X).nanmean().backward()
                fopt.step()

        f.eval()
        X, Y = env.test()
        for i in range(35):
            with torch.no_grad():
                yield {'l1':loss(f(torch.nan_to_num(X)),X).nanmean()}

def make_envs(X, Y, G):
    X, Y, G = torch.tensor(X).float(), torch.tensor(Y).float(), torch.tensor(G)
    all_equal = set()
    envs = []
    for rng in range(1):
        for g in sorted(set(G.tolist())):
            if (g==G).sum() < 50: continue
            trn_x,trn_y = X[g!=G],Y[g!=G]
            tst_x,tst_y = X[g==G],Y[g==G]

            if len(set(tst_y.tolist())) == 1:
                all_equal.add(g)
            else:
                envs.append(MyEnvironment(trn_x,trn_y,tst_x,tst_y,g,rng))
    if all_equal: print(f"All Equal, no environment added for {sorted(all_equal)}") 
    return envs

lrns = [ FeedForward(specs=[], rng=1) ]

vals = lambda x: [
    MyEvaluator((len(x),20,'l','r',20,'l','r',len(x)),2),
    MyEvaluator((len(x),20,'l','r',20,'l','r',len(x)),4),
    MyEvaluator((len(x),20,'l','r',20,'l','r',len(x)),8),
    MyEvaluator((len(x),25,'l','r',25,'l','r',len(x)),2),
    MyEvaluator((len(x),25,'l','r',25,'l','r',len(x)),4),
    MyEvaluator((len(x),25,'l','r',25,'l','r',len(x)),8),
]

envs = make_envs(X3,Y2,G)
cb.Experiment(envs,lrns,vals(X3[0])).run('../logs/outae4.log',processes=35,quiet=True)


All Equal, no environment added for [2, 25, 61, 82, 87, 89, 99, 103, 111, 117, 146, 196, 203, 227, 246, 311, 319, 357, 377, 390, 451, 455]
{'Learners': 1, 'Environments': 120, 'Interactions': 23870}


In [ ]:
import coba as cb

class FeedForward(torch.nn.Sequential):
    """A Generic implementation of Feedforward Neural Network"""

    class SkipModule(torch.nn.Module):
        def __init__(self, layers):
            super().__init__()
            self.layers = layers
        def forward(self,X):
            return X + self.layers(X)

    def make_layer(self,curr_dim,spec):
        if isinstance(spec,float):
            return torch.nn.Dropout(spec), curr_dim
        if curr_dim is None and isinstance(spec,int):
            return None, spec
        if isinstance(spec,int):
            return torch.nn.Linear(curr_dim,spec),spec
        if spec == 'r':
            return torch.nn.ReLU(),curr_dim
        if spec == 'l':
            return torch.nn.LayerNorm(curr_dim),curr_dim
        if spec == 'b':
            return torch.nn.BatchNorm1d(curr_dim), curr_dim
        if spec == 's':
            return torch.nn.Sigmoid(),curr_dim
        if isinstance(spec,list):                
            return FeedForward.SkipModule(FeedForward([curr_dim] + spec)), curr_dim
        raise Exception("Bad Layer")

    def __init__(self, specs, rng=1):
        """Instantiate a Feedfoward network according to specifications.

        Args:
            specs: A sequence of layer specifications as follows:
                -1 -- replaced with the input feature width
                <int> -- a LinearLayer with output width equal to <int>
                [0,1] -- a Dropout layer with the given probability
                'l' -- a LayerNorm
                'b' -- a BatchNorm1d
                'r' -- a ReLU layer
                's' -- a Sigmoid layer
                [] -- a skip layer with the given specifications
        """

        torch.manual_seed(rng)
        layers,width = [],None
        for spec in specs:
            layer,width = self.make_layer(width,spec)
            if layer: layers.append(layer)
        super().__init__(*(layers or [torch.nn.Identity()]))
        self.params = {"specs": specs, "rng": rng }

class MyEnvironment:
    def __init__(self, train_X, train_Y, test_X, test_Y, g, rng):
        self.params = {'pid': g, 'rng': rng}
        self.train_X = train_X
        self.train_Y = train_Y.float()
        self.test_X = test_X
        self.test_Y = test_Y.float()

    def ssl(self,neg,sr):

        by_class  = {y: self.train_X[(self.train_Y==y)].tolist() for y in set(self.train_Y.tolist())}
        rng = cb.CobaRandom(self.params['rng'])

        def choose_unique(items,item):
            while items[i:=rng.randint(0,len(items)-1)]==item:
                pass
            return items[i]

        def choose_n(items,n):
            indexes = set()
            while True:
                indexes.add(rng.randint(0,len(items)-1))
                if len(indexes)==n: return [items[i] for i in indexes]

        for _ in range(sr):
            anchor, positive, negative = [], [], []
            for x,y in zip(self.train_X.tolist(),self.train_Y.tolist()):
                anchor.append(x)
                positive.append(choose_unique(by_class[y],x))
                negative.append(choose_n(by_class[1-y],neg))

            yield torch.tensor(anchor).float(), torch.tensor(positive).float(), torch.tensor(negative).float()

    def train(self):
        return self.train_X, self.train_Y

    def test(self):
        rng_indexes = cb.CobaRandom(self.params['rng']).shuffle(range(len(self.test_X)))
        return self.test_X[rng_indexes,:], self.test_Y[rng_indexes]

class MyEvaluator:
    def __init__(self, s1, ws):
        self.s1 = s1
        self.ws = ws
        self.params = {'s1': s1, 'ws':ws }

    def evaluate(self, env, lrn):        
        torch.set_num_threads(1)
        torch.manual_seed(1)
        
        X, Y = env.train()

        loss = torch.nn.L1Loss(reduction='none')
        torch_loader  = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X),batch_size=16,shuffle=True)

        f = FeedForward(self.s1)
        fopt = COCOB(f.parameters())
        f.train()

        for _ in range(self.ws):
            for (_X,) in torch_loader:
                fopt.zero_grad()
                loss(f(torch.nan_to_num(_X)),_X).nanmean().backward()
                fopt.step()

        f.eval()
        X, Y = env.test()
        for i in range(35):
            with torch.no_grad():
                yield {'l1':loss(f(torch.nan_to_num(X)),X).nanmean()}

def make_envs(X, Y, G):
    X, Y, G = torch.tensor(X).float(), torch.tensor(Y).float(), torch.tensor(G)
    all_equal = set()
    envs = []
    for rng in range(1):
        for g in sorted(set(G.tolist())):
            if (g==G).sum() < 50: continue
            trn_x,trn_y = X[g!=G],Y[g!=G]
            tst_x,tst_y = X[g==G],Y[g==G]

            if len(set(tst_y.tolist())) == 1:
                all_equal.add(g)
            else:
                envs.append(MyEnvironment(trn_x,trn_y,tst_x,tst_y,g,rng))
    if all_equal: print(f"All Equal, no environment added for {sorted(all_equal)}") 
    return envs

lrns = [ FeedForward(specs=[], rng=1) ]

vals = lambda x: [
    MyEvaluator((len(x),50,'l','r',25,'l','r',50,'l','r',len(x)),8),
    MyEvaluator((len(x),25,'l','r',len(x)),8),
    MyEvaluator((len(x),25,'l','r',25,'l','r',len(x)),8),
]

envs = make_envs(X3,Y2,G)
cb.Experiment(envs,lrns,vals(X3[0])).run('../logs/outae5.log',processes=35,quiet=True)


All Equal, no environment added for [2, 25, 61, 82, 87, 89, 99, 103, 111, 117, 146, 196, 203, 227, 246, 311, 319, 357, 377, 390, 451, 455]
{'Learners': 1, 'Environments': 44, 'Interactions': 3360}


In [ ]:
import coba as cb

class FeedForward(torch.nn.Sequential):
    """A Generic implementation of Feedforward Neural Network"""

    class SkipModule(torch.nn.Module):
        def __init__(self, layers):
            super().__init__()
            self.layers = layers
        def forward(self,X):
            return X + self.layers(X)

    def make_layer(self,curr_dim,spec):
        if isinstance(spec,float):
            return torch.nn.Dropout(spec), curr_dim
        if curr_dim is None and isinstance(spec,int):
            return None, spec
        if isinstance(spec,int):
            return torch.nn.Linear(curr_dim,spec),spec
        if spec == 'r':
            return torch.nn.ReLU(),curr_dim
        if spec == 'l':
            return torch.nn.LayerNorm(curr_dim),curr_dim
        if spec == 'b':
            return torch.nn.BatchNorm1d(curr_dim), curr_dim
        if spec == 's':
            return torch.nn.Sigmoid(),curr_dim
        if isinstance(spec,list):                
            return FeedForward.SkipModule(FeedForward([curr_dim] + spec)), curr_dim
        raise Exception("Bad Layer")

    def __init__(self, specs, rng=1):
        """Instantiate a Feedfoward network according to specifications.

        Args:
            specs: A sequence of layer specifications as follows:
                -1 -- replaced with the input feature width
                <int> -- a LinearLayer with output width equal to <int>
                [0,1] -- a Dropout layer with the given probability
                'l' -- a LayerNorm
                'b' -- a BatchNorm1d
                'r' -- a ReLU layer
                's' -- a Sigmoid layer
                [] -- a skip layer with the given specifications
        """

        torch.manual_seed(rng)
        layers,width = [],None
        for spec in specs:
            layer,width = self.make_layer(width,spec)
            if layer: layers.append(layer)
        super().__init__(*(layers or [torch.nn.Identity()]))
        self.params = {"specs": specs, "rng": rng }

class MyEnvironment:
    def __init__(self, train_X, train_Y, test_X, test_Y, g, rng):
        self.params = {'pid': g, 'rng': rng}
        self.train_X = train_X
        self.train_Y = train_Y.float()
        self.test_X = test_X
        self.test_Y = test_Y.float()

    def ssl(self,neg,sr):

        by_class  = {y: self.train_X[(self.train_Y==y)].tolist() for y in set(self.train_Y.tolist())}
        rng = cb.CobaRandom(self.params['rng'])

        def choose_unique(items,item):
            while items[i:=rng.randint(0,len(items)-1)]==item:
                pass
            return items[i]

        def choose_n(items,n):
            indexes = set()
            while True:
                indexes.add(rng.randint(0,len(items)-1))
                if len(indexes)==n: return [items[i] for i in indexes]

        for _ in range(sr):
            anchor, positive, negative = [], [], []
            for x,y in zip(self.train_X.tolist(),self.train_Y.tolist()):
                anchor.append(x)
                positive.append(choose_unique(by_class[y],x))
                negative.append(choose_n(by_class[1-y],neg))

            yield torch.tensor(anchor).float(), torch.tensor(positive).float(), torch.tensor(negative).float()

    def train(self):
        return self.train_X, self.train_Y

    def test(self):
        rng_indexes = cb.CobaRandom(self.params['rng']).shuffle(range(len(self.test_X)))
        return self.test_X[rng_indexes,:], self.test_Y[rng_indexes]

class MyEvaluator:
    def __init__(self, s1, ws):
        self.s1 = s1
        self.ws = ws
        self.params = {'s1': s1, 'ws':ws }

    def evaluate(self, env, lrn):        
        torch.set_num_threads(1)
        torch.manual_seed(1)
        
        X, Y = env.train()

        loss = torch.nn.L1Loss(reduction='none')
        torch_loader  = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X),batch_size=16,shuffle=True)

        f = FeedForward(self.s1)
        fopt = COCOB(f.parameters())
        f.train()

        for _ in range(self.ws):
            for (_X,) in torch_loader:
                fopt.zero_grad()
                loss(f(torch.nan_to_num(_X)),_X).nanmean().backward()
                fopt.step()

        f.eval()
        X, Y = env.test()
        for i in range(35):
            with torch.no_grad():
                yield {'l1':loss(f(torch.nan_to_num(X)),X).nanmean()}

def make_envs(X, Y, G):
    X, Y, G = torch.tensor(X).float(), torch.tensor(Y).float(), torch.tensor(G)
    all_equal = set()
    envs = []
    for rng in range(1):
        for g in sorted(set(G.tolist())):
            if (g==G).sum() < 50: continue
            trn_x,trn_y = X[g!=G],Y[g!=G]
            tst_x,tst_y = X[g==G],Y[g==G]

            if len(set(tst_y.tolist())) == 1:
                all_equal.add(g)
            else:
                envs.append(MyEnvironment(trn_x,trn_y,tst_x,tst_y,g,rng))
    if all_equal: print(f"All Equal, no environment added for {sorted(all_equal)}") 
    return envs

lrns = [ FeedForward(specs=[], rng=1) ]

vals = lambda x: [
    MyEvaluator((len(x),25,'l','r',len(x)),8),
    MyEvaluator((len(x),25,'l','r',len(x)),32),
    MyEvaluator((len(x),25,'l','r',25,'l','r',len(x)),32),
]

envs = make_envs(X3,Y2,G)
cb.Experiment(envs,lrns,vals(X3[0])).run('../logs/outae6.log',processes=35,quiet=True)


All Equal, no environment added for [2, 25, 61, 82, 87, 89, 99, 103, 111, 117, 146, 196, 203, 227, 246, 311, 319, 357, 377, 390, 451, 455]
{'Learners': 1, 'Environments': 124, 'Interactions': 11725}


In [ ]:
import coba as cb

class FeedForward(torch.nn.Sequential):
    """A Generic implementation of Feedforward Neural Network"""

    class SkipModule(torch.nn.Module):
        def __init__(self, layers):
            super().__init__()
            self.layers = layers
        def forward(self,X):
            return X + self.layers(X)

    def make_layer(self,curr_dim,spec):
        if isinstance(spec,float):
            return torch.nn.Dropout(spec), curr_dim
        if curr_dim is None and isinstance(spec,int):
            return None, spec
        if isinstance(spec,int):
            return torch.nn.Linear(curr_dim,spec),spec
        if spec == 'r':
            return torch.nn.ReLU(),curr_dim
        if spec == 'l':
            return torch.nn.LayerNorm(curr_dim),curr_dim
        if spec == 'b':
            return torch.nn.BatchNorm1d(curr_dim), curr_dim
        if spec == 's':
            return torch.nn.Sigmoid(),curr_dim
        if isinstance(spec,list):                
            return FeedForward.SkipModule(FeedForward([curr_dim] + spec)), curr_dim
        raise Exception("Bad Layer")

    def __init__(self, specs, rng=1):
        """Instantiate a Feedfoward network according to specifications.

        Args:
            specs: A sequence of layer specifications as follows:
                -1 -- replaced with the input feature width
                <int> -- a LinearLayer with output width equal to <int>
                [0,1] -- a Dropout layer with the given probability
                'l' -- a LayerNorm
                'b' -- a BatchNorm1d
                'r' -- a ReLU layer
                's' -- a Sigmoid layer
                [] -- a skip layer with the given specifications
        """

        torch.manual_seed(rng)
        layers,width = [],None
        for spec in specs:
            layer,width = self.make_layer(width,spec)
            if layer: layers.append(layer)
        super().__init__(*(layers or [torch.nn.Identity()]))
        self.params = {"specs": specs, "rng": rng }

class MyEnvironment:
    def __init__(self, train_X, train_Y, test_X, test_Y, g, rng):
        self.params = {'pid': g, 'rng': rng}
        self.train_X = train_X
        self.train_Y = train_Y.float()
        self.test_X = test_X
        self.test_Y = test_Y.float()

    def ssl(self,neg,sr):

        by_class  = {y: self.train_X[(self.train_Y==y)].tolist() for y in set(self.train_Y.tolist())}
        rng = cb.CobaRandom(self.params['rng'])

        def choose_unique(items,item):
            while items[i:=rng.randint(0,len(items)-1)]==item:
                pass
            return items[i]

        def choose_n(items,n):
            indexes = set()
            while True:
                indexes.add(rng.randint(0,len(items)-1))
                if len(indexes)==n: return [items[i] for i in indexes]

        for _ in range(sr):
            anchor, positive, negative = [], [], []
            for x,y in zip(self.train_X.tolist(),self.train_Y.tolist()):
                anchor.append(x)
                positive.append(choose_unique(by_class[y],x))
                negative.append(choose_n(by_class[1-y],neg))

            yield torch.tensor(anchor).float(), torch.tensor(positive).float(), torch.tensor(negative).float()

    def train(self):
        return self.train_X, self.train_Y

    def test(self):
        rng_indexes = cb.CobaRandom(self.params['rng']).shuffle(range(len(self.test_X)))
        return self.test_X[rng_indexes,:], self.test_Y[rng_indexes]

class MyEvaluator:
    def __init__(self, s1, ws):
        self.s1 = s1
        self.ws = ws
        self.params = {'s1': s1, 'ws':ws }

    def evaluate(self, env, lrn):        
        torch.set_num_threads(1)
        torch.manual_seed(1)
        
        X, Y = env.train()

        loss = torch.nn.L1Loss(reduction='none')
        torch_loader  = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X),batch_size=16,shuffle=True)

        f = FeedForward(self.s1)
        fopt = COCOB(f.parameters())
        f.train()

        for _ in range(self.ws):
            for (_X,) in torch_loader:
                fopt.zero_grad()
                loss(f(torch.nan_to_num(_X)),_X).nanmean().backward()
                fopt.step()

        f.eval()
        X, Y = env.test()
        for i in range(35):
            with torch.no_grad():
                yield {'l1':loss(f(torch.nan_to_num(X)),X).nanmean()}

def make_envs(X, Y, G):
    X, Y, G = torch.tensor(X).float(), torch.tensor(Y).float(), torch.tensor(G)
    all_equal = set()
    envs = []
    for rng in range(1):
        for g in sorted(set(G.tolist())):
            if (g==G).sum() < 50: continue
            trn_x,trn_y = X[g!=G],Y[g!=G]
            tst_x,tst_y = X[g==G],Y[g==G]

            if len(set(tst_y.tolist())) == 1:
                all_equal.add(g)
            else:
                envs.append(MyEnvironment(trn_x,trn_y,tst_x,tst_y,g,rng))
    if all_equal: print(f"All Equal, no environment added for {sorted(all_equal)}") 
    return envs

lrns = [ FeedForward(specs=[], rng=1) ]

vals = lambda x: [
    MyEvaluator((len(x),20,'l','r',len(x)),64),
    MyEvaluator((len(x),25,'l','r',len(x)),32),
    MyEvaluator((len(x),25,'l','r',len(x)),64),
    MyEvaluator((len(x),30,'l','r',len(x)),64),
]

envs = make_envs(X3,Y2,G)
cb.Experiment(envs,lrns,vals(X3[0])).run('../logs/outae7.log',processes=35,quiet=True)


All Equal, no environment added for [2, 25, 61, 82, 87, 89, 99, 103, 111, 117, 146, 196, 203, 227, 246, 311, 319, 357, 377, 390, 451, 455]
{'Learners': 1, 'Environments': 28, 'Interactions': 2660}


In [ ]:
import coba as cb

class FeedForward(torch.nn.Sequential):
    """A Generic implementation of Feedforward Neural Network"""

    class SkipModule(torch.nn.Module):
        def __init__(self, layers):
            super().__init__()
            self.layers = layers
        def forward(self,X):
            return X + self.layers(X)

    def make_layer(self,curr_dim,spec):
        if isinstance(spec,float):
            return torch.nn.Dropout(spec), curr_dim
        if curr_dim is None and isinstance(spec,int):
            return None, spec
        if isinstance(spec,int):
            return torch.nn.Linear(curr_dim,spec),spec
        if spec == 'r':
            return torch.nn.ReLU(),curr_dim
        if spec == 'l':
            return torch.nn.LayerNorm(curr_dim),curr_dim
        if spec == 'b':
            return torch.nn.BatchNorm1d(curr_dim), curr_dim
        if spec == 's':
            return torch.nn.Sigmoid(),curr_dim
        if isinstance(spec,list):                
            return FeedForward.SkipModule(FeedForward([curr_dim] + spec)), curr_dim
        raise Exception("Bad Layer")

    def __init__(self, specs, rng=1):
        """Instantiate a Feedfoward network according to specifications.

        Args:
            specs: A sequence of layer specifications as follows:
                -1 -- replaced with the input feature width
                <int> -- a LinearLayer with output width equal to <int>
                [0,1] -- a Dropout layer with the given probability
                'l' -- a LayerNorm
                'b' -- a BatchNorm1d
                'r' -- a ReLU layer
                's' -- a Sigmoid layer
                [] -- a skip layer with the given specifications
        """

        torch.manual_seed(rng)
        layers,width = [],None
        for spec in specs:
            layer,width = self.make_layer(width,spec)
            if layer: layers.append(layer)
        super().__init__(*(layers or [torch.nn.Identity()]))
        self.params = {"specs": specs, "rng": rng }

class MyEnvironment:
    def __init__(self, train_X, train_Y, test_X, test_Y, g, rng):
        self.params = {'pid': g, 'rng': rng}
        self.train_X = train_X
        self.train_Y = train_Y.float()
        self.test_X = test_X
        self.test_Y = test_Y.float()

    def ssl(self,neg,sr):

        by_class  = {y: self.train_X[(self.train_Y==y)].tolist() for y in set(self.train_Y.tolist())}
        rng = cb.CobaRandom(self.params['rng'])

        def choose_unique(items,item):
            while items[i:=rng.randint(0,len(items)-1)]==item:
                pass
            return items[i]

        def choose_n(items,n):
            indexes = set()
            while True:
                indexes.add(rng.randint(0,len(items)-1))
                if len(indexes)==n: return [items[i] for i in indexes]

        for _ in range(sr):
            anchor, positive, negative = [], [], []
            for x,y in zip(self.train_X.tolist(),self.train_Y.tolist()):
                anchor.append(x)
                positive.append(choose_unique(by_class[y],x))
                negative.append(choose_n(by_class[1-y],neg))

            yield torch.tensor(anchor).float(), torch.tensor(positive).float(), torch.tensor(negative).float()

    def train(self):
        return self.train_X, self.train_Y

    def test(self):
        rng_indexes = cb.CobaRandom(self.params['rng']).shuffle(range(len(self.test_X)))
        return self.test_X[rng_indexes,:], self.test_Y[rng_indexes]

class MyEvaluator:
    def __init__(self, s1, ws):
        self.s1 = s1
        self.ws = ws
        self.params = {'s1': s1, 'ws':ws }

    def evaluate(self, env, lrn):        
        torch.set_num_threads(1)
        torch.manual_seed(1)
        
        X, Y = env.train()

        loss = torch.nn.L1Loss(reduction='none')
        torch_loader  = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X),batch_size=16,shuffle=True)

        f = FeedForward(self.s1)
        fopt = COCOB(f.parameters())
        f.train()

        for _ in range(self.ws):
            for (_X,) in torch_loader:
                fopt.zero_grad()
                loss(f(torch.nan_to_num(_X)),_X).nanmean().backward()
                fopt.step()

        f.eval()
        X, Y = env.test()
        for i in range(35):
            with torch.no_grad():
                yield {'l1':loss(f(torch.nan_to_num(X)),X).nanmean()}

def make_envs(X, Y, G):
    X, Y, G = torch.tensor(X).float(), torch.tensor(Y).float(), torch.tensor(G)
    all_equal = set()
    envs = []
    for rng in range(1):
        for g in sorted(set(G.tolist())):
            if (g==G).sum() < 50: continue
            trn_x,trn_y = X[g!=G],Y[g!=G]
            tst_x,tst_y = X[g==G],Y[g==G]

            if len(set(tst_y.tolist())) == 1:
                all_equal.add(g)
            else:
                envs.append(MyEnvironment(trn_x,trn_y,tst_x,tst_y,g,rng))
    if all_equal: print(f"All Equal, no environment added for {sorted(all_equal)}") 
    return envs

lrns = [ FeedForward(specs=[], rng=1) ]

vals = lambda x: [
    MyEvaluator((len(x),30,'l','r',len(x)), 8),
    MyEvaluator((len(x),30,'l','r',len(x)),16),
    MyEvaluator((len(x),30,'l','r',len(x)),32),
    MyEvaluator((len(x),30,'l','r',30,'l','r',len(x)),32),
    MyEvaluator((len(x),45,'l','r',len(x)), 8),
    MyEvaluator((len(x),45,'l','r',len(x)),16),
    MyEvaluator((len(x),45,'l','r',len(x)),32),
]

envs = make_envs(X3,Y2,G)
cb.Experiment(envs,lrns,vals(X3[0])).run('../logs/outae8.log',processes=35,quiet=True)


All Equal, no environment added for [2, 25, 61, 82, 87, 89, 99, 103, 111, 117, 146, 196, 203, 227, 246, 311, 319, 357, 377, 390, 451, 455]
{'Learners': 1, 'Environments': 26, 'Interactions': 5040}


In [ ]:
import coba as cb

class FeedForward(torch.nn.Sequential):
    """A Generic implementation of Feedforward Neural Network"""

    class SkipModule(torch.nn.Module):
        def __init__(self, layers):
            super().__init__()
            self.layers = layers
        def forward(self,X):
            return X + self.layers(X)

    def make_layer(self,curr_dim,spec):
        if isinstance(spec,float):
            return torch.nn.Dropout(spec), curr_dim
        if curr_dim is None and isinstance(spec,int):
            return None, spec
        if isinstance(spec,int):
            return torch.nn.Linear(curr_dim,spec),spec
        if spec == 'r':
            return torch.nn.ReLU(),curr_dim
        if spec == 'l':
            return torch.nn.LayerNorm(curr_dim),curr_dim
        if spec == 'b':
            return torch.nn.BatchNorm1d(curr_dim), curr_dim
        if spec == 's':
            return torch.nn.Sigmoid(),curr_dim
        if isinstance(spec,list):                
            return FeedForward.SkipModule(FeedForward([curr_dim] + spec)), curr_dim
        raise Exception("Bad Layer")

    def __init__(self, specs, rng=1):
        """Instantiate a Feedfoward network according to specifications.

        Args:
            specs: A sequence of layer specifications as follows:
                -1 -- replaced with the input feature width
                <int> -- a LinearLayer with output width equal to <int>
                [0,1] -- a Dropout layer with the given probability
                'l' -- a LayerNorm
                'b' -- a BatchNorm1d
                'r' -- a ReLU layer
                's' -- a Sigmoid layer
                [] -- a skip layer with the given specifications
        """

        torch.manual_seed(rng)
        layers,width = [],None
        for spec in specs:
            layer,width = self.make_layer(width,spec)
            if layer: layers.append(layer)
        super().__init__(*(layers or [torch.nn.Identity()]))
        self.params = {"specs": specs, "rng": rng }

class MyEnvironment:
    def __init__(self, train_X, train_Y, test_X, test_Y, g, rng):
        self.params = {'pid': g, 'rng': rng}
        self.train_X = train_X
        self.train_Y = train_Y.float()
        self.test_X = test_X
        self.test_Y = test_Y.float()

    def ssl(self,neg,sr):

        by_class  = {y: self.train_X[(self.train_Y==y)].tolist() for y in set(self.train_Y.tolist())}
        rng = cb.CobaRandom(self.params['rng'])

        def choose_unique(items,item):
            while items[i:=rng.randint(0,len(items)-1)]==item:
                pass
            return items[i]

        def choose_n(items,n):
            indexes = set()
            while True:
                indexes.add(rng.randint(0,len(items)-1))
                if len(indexes)==n: return [items[i] for i in indexes]

        for _ in range(sr):
            anchor, positive, negative = [], [], []
            for x,y in zip(self.train_X.tolist(),self.train_Y.tolist()):
                anchor.append(x)
                positive.append(choose_unique(by_class[y],x))
                negative.append(choose_n(by_class[1-y],neg))

            yield torch.tensor(anchor).float(), torch.tensor(positive).float(), torch.tensor(negative).float()

    def train(self):
        return self.train_X, self.train_Y

    def test(self):
        rng_indexes = cb.CobaRandom(self.params['rng']).shuffle(range(len(self.test_X)))
        return self.test_X[rng_indexes,:], self.test_Y[rng_indexes]

class MyEvaluator:
    def __init__(self, s1, ws):
        self.s1 = s1
        self.ws = ws
        self.params = {'s1': s1, 'ws':ws }

    def evaluate(self, env, lrn):        
        torch.set_num_threads(1)
        torch.manual_seed(1)
        
        X, Y = env.train()

        loss = torch.nn.L1Loss(reduction='none')
        torch_loader  = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X),batch_size=16,shuffle=True)

        f = FeedForward(self.s1)
        fopt = COCOB(f.parameters())
        f.train()

        for _ in range(self.ws):
            for (_X,) in torch_loader:
                fopt.zero_grad()
                loss(f(torch.nan_to_num(_X)),_X).nanmean().backward()
                fopt.step()

        f.eval()
        X, Y = env.test()
        for i in range(35):
            with torch.no_grad():
                yield {'l1':loss(f(torch.nan_to_num(X)),X).nanmean()}

def make_envs(X, Y, G):
    X, Y, G = torch.tensor(X).float(), torch.tensor(Y).float(), torch.tensor(G)
    all_equal = set()
    envs = []
    for rng in range(1):
        for g in sorted(set(G.tolist())):
            if (g==G).sum() < 50: continue
            trn_x,trn_y = X[g!=G],Y[g!=G]
            tst_x,tst_y = X[g==G],Y[g==G]

            if len(set(tst_y.tolist())) == 1:
                all_equal.add(g)
            else:
                envs.append(MyEnvironment(trn_x,trn_y,tst_x,tst_y,g,rng))
    if all_equal: print(f"All Equal, no environment added for {sorted(all_equal)}") 
    return envs

lrns = [ FeedForward(specs=[], rng=1) ]

vals = lambda x: [
    MyEvaluator((len(x),.2,30,'l','r',len(x)), 8),
    MyEvaluator((len(x),.2,30,'l','r',len(x)),16),
    MyEvaluator((len(x),.2,30,'l','r',len(x)),32),
    MyEvaluator((len(x),.2,30,'l','r',30,'l','r',len(x)),32),
    MyEvaluator((len(x),.2,45,'l','r',len(x)), 8),
    MyEvaluator((len(x),.2,45,'l','r',len(x)),16),
    MyEvaluator((len(x),.2,45,'l','r',len(x)),32),
]

envs = make_envs(X3,Y2,G)
cb.Experiment(envs,lrns,vals(X3[0])).run('../logs/outae8.log',processes=35,quiet=True)


All Equal, no environment added for [2, 25, 61, 82, 87, 89, 99, 103, 111, 117, 146, 196, 203, 227, 246, 311, 319, 357, 377, 390, 451, 455]
{'Learners': 1, 'Environments': 26, 'Interactions': 5040}


Background process 748598 failed unexpectedly with exit code 1.
Background process 748418 failed unexpectedly with exit code 1.
Background process 748553 failed unexpectedly with exit code 1.
Background process 748600 failed unexpectedly with exit code 1.
Background process 748415 failed unexpectedly with exit code 1.
Background process 748469 failed unexpectedly with exit code 1.
Background process 748368 failed unexpectedly with exit code 1.
Background process 748594 failed unexpectedly with exit code 1.
Background process 748596 failed unexpectedly with exit code 1.
Background process 748420 failed unexpectedly with exit code 1.
Background process 748471 failed unexpectedly with exit code 1.
Background process 748360 failed unexpectedly with exit code 1.
Background process 748463 failed unexpectedly with exit code 1.
Background process 748604 failed unexpectedly with exit code 1.
Background process 748512 failed unexpectedly with exit code 1.
Background process 748465 failed unexpec

In [ ]:
import coba as cb

class FeedForward(torch.nn.Sequential):
    """A Generic implementation of Feedforward Neural Network"""

    class SkipModule(torch.nn.Module):
        def __init__(self, layers):
            super().__init__()
            self.layers = layers
        def forward(self,X):
            return X + self.layers(X)

    def make_layer(self,curr_dim,spec):
        if isinstance(spec,float):
            return torch.nn.Dropout(spec), curr_dim
        if curr_dim is None and isinstance(spec,int):
            return None, spec
        if isinstance(spec,int):
            return torch.nn.Linear(curr_dim,spec),spec
        if spec == 'r':
            return torch.nn.ReLU(),curr_dim
        if spec == 'l':
            return torch.nn.LayerNorm(curr_dim),curr_dim
        if spec == 'b':
            return torch.nn.BatchNorm1d(curr_dim), curr_dim
        if spec == 's':
            return torch.nn.Sigmoid(),curr_dim
        if isinstance(spec,list):                
            return FeedForward.SkipModule(FeedForward([curr_dim] + spec)), curr_dim
        raise Exception("Bad Layer")

    def __init__(self, specs, rng=1):
        """Instantiate a Feedfoward network according to specifications.

        Args:
            specs: A sequence of layer specifications as follows:
                -1 -- replaced with the input feature width
                <int> -- a LinearLayer with output width equal to <int>
                [0,1] -- a Dropout layer with the given probability
                'l' -- a LayerNorm
                'b' -- a BatchNorm1d
                'r' -- a ReLU layer
                's' -- a Sigmoid layer
                [] -- a skip layer with the given specifications
        """

        torch.manual_seed(rng)
        layers,width = [],None
        for spec in specs:
            layer,width = self.make_layer(width,spec)
            if layer: layers.append(layer)
        super().__init__(*(layers or [torch.nn.Identity()]))
        self.params = {"specs": specs, "rng": rng }

class MyEnvironment:
    def __init__(self, train_X, train_Y, test_X, test_Y, g, rng):
        self.params = {'pid': g, 'rng': rng}
        self.train_X = train_X
        self.train_Y = train_Y.float()
        self.test_X = test_X
        self.test_Y = test_Y.float()

    def ssl(self,neg,sr):

        by_class  = {y: self.train_X[(self.train_Y==y)].tolist() for y in set(self.train_Y.tolist())}
        rng = cb.CobaRandom(self.params['rng'])

        def choose_unique(items,item):
            while items[i:=rng.randint(0,len(items)-1)]==item:
                pass
            return items[i]

        def choose_n(items,n):
            indexes = set()
            while True:
                indexes.add(rng.randint(0,len(items)-1))
                if len(indexes)==n: return [items[i] for i in indexes]

        for _ in range(sr):
            anchor, positive, negative = [], [], []
            for x,y in zip(self.train_X.tolist(),self.train_Y.tolist()):
                anchor.append(x)
                positive.append(choose_unique(by_class[y],x))
                negative.append(choose_n(by_class[1-y],neg))

            yield torch.tensor(anchor).float(), torch.tensor(positive).float(), torch.tensor(negative).float()

    def train(self):
        return self.train_X, self.train_Y

    def test(self):
        rng_indexes = cb.CobaRandom(self.params['rng']).shuffle(range(len(self.test_X)))
        return self.test_X[rng_indexes,:], self.test_Y[rng_indexes]

class MyEvaluator:
    def __init__(self, s1, ws):
        self.s1 = s1
        self.ws = ws
        self.params = {'s1': s1, 'ws':ws }

    def evaluate(self, env, lrn):        
        torch.set_num_threads(1)
        torch.manual_seed(1)
        
        X, Y = env.train()

        loss = torch.nn.L1Loss(reduction='none')
        torch_loader  = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X),batch_size=16,shuffle=True)

        f = FeedForward(self.s1)
        fopt = COCOB(f.parameters())
        f.train()

        for _ in range(self.ws):
            for (_X,) in torch_loader:
                fopt.zero_grad()
                loss(f(torch.nan_to_num(_X)),_X).nanmean().backward()
                fopt.step()

        f.eval()
        X, Y = env.test()
        for i in range(35):
            with torch.no_grad():
                yield {'l1':loss(f(torch.nan_to_num(X)),X).nanmean()}

def make_envs(X, Y, G):
    X, Y, G = torch.tensor(X).float(), torch.tensor(Y).float(), torch.tensor(G)
    all_equal = set()
    envs = []
    for rng in range(1):
        for g in sorted(set(G.tolist())):
            if (g==G).sum() < 50: continue
            trn_x,trn_y = X[g!=G],Y[g!=G]
            tst_x,tst_y = X[g==G],Y[g==G]

            if len(set(tst_y.tolist())) == 1:
                all_equal.add(g)
            else:
                envs.append(MyEnvironment(trn_x,trn_y,tst_x,tst_y,g,rng))
    if all_equal: print(f"All Equal, no environment added for {sorted(all_equal)}") 
    return envs

lrns = [ FeedForward(specs=[], rng=1) ]

vals = lambda x: [
    MyEvaluator((len(x),.2,30,'l','r',len(x)), 8),
    MyEvaluator((len(x),.2,30,'l','r',len(x)),16),
    MyEvaluator((len(x),.2,30,'l','r',len(x)),32),
    MyEvaluator((len(x),.2,30,'l','r',30,'l','r',len(x)),32),
    MyEvaluator((len(x),.2,45,'l','r',len(x)), 8),
    MyEvaluator((len(x),.2,45,'l','r',len(x)),16),
    MyEvaluator((len(x),.2,45,'l','r',len(x)),32),
]

envs = make_envs(X3,Y2,G)
cb.Experiment(envs,lrns,vals(X3[0])).run('../logs/outae9.log',processes=35,quiet=True)


All Equal, no environment added for [2, 25, 61, 82, 87, 89, 99, 103, 111, 117, 146, 196, 203, 227, 246, 311, 319, 357, 377, 390, 451, 455]
{'Learners': 1, 'Environments': 15, 'Interactions': 2345}


In [ ]:
import coba as cb

class FeedForward(torch.nn.Sequential):
    """A Generic implementation of Feedforward Neural Network"""

    class SkipModule(torch.nn.Module):
        def __init__(self, layers):
            super().__init__()
            self.layers = layers
        def forward(self,X):
            return X + self.layers(X)

    def make_layer(self,curr_dim,spec):
        if isinstance(spec,float):
            return torch.nn.Dropout(spec), curr_dim
        if curr_dim is None and isinstance(spec,int):
            return None, spec
        if isinstance(spec,int):
            return torch.nn.Linear(curr_dim,spec),spec
        if spec == 'r':
            return torch.nn.ReLU(),curr_dim
        if spec == 'l':
            return torch.nn.LayerNorm(curr_dim),curr_dim
        if spec == 'b':
            return torch.nn.BatchNorm1d(curr_dim), curr_dim
        if spec == 's':
            return torch.nn.Sigmoid(),curr_dim
        if isinstance(spec,list):                
            return FeedForward.SkipModule(FeedForward([curr_dim] + spec)), curr_dim
        raise Exception("Bad Layer")

    def __init__(self, specs, rng=1):
        """Instantiate a Feedfoward network according to specifications.

        Args:
            specs: A sequence of layer specifications as follows:
                -1 -- replaced with the input feature width
                <int> -- a LinearLayer with output width equal to <int>
                [0,1] -- a Dropout layer with the given probability
                'l' -- a LayerNorm
                'b' -- a BatchNorm1d
                'r' -- a ReLU layer
                's' -- a Sigmoid layer
                [] -- a skip layer with the given specifications
        """

        torch.manual_seed(rng)
        layers,width = [],None
        for spec in specs:
            layer,width = self.make_layer(width,spec)
            if layer: layers.append(layer)
        super().__init__(*(layers or [torch.nn.Identity()]))
        self.params = {"specs": specs, "rng": rng }

class MyEnvironment:
    def __init__(self, train_X, train_Y, test_X, test_Y, g, rng):
        self.params = {'pid': g, 'rng': rng}
        self.train_X = train_X
        self.train_Y = train_Y.float()
        self.test_X = test_X
        self.test_Y = test_Y.float()

    def ssl(self,neg,sr):

        by_class  = {y: self.train_X[(self.train_Y==y)].tolist() for y in set(self.train_Y.tolist())}
        rng = cb.CobaRandom(self.params['rng'])

        def choose_unique(items,item):
            while items[i:=rng.randint(0,len(items)-1)]==item:
                pass
            return items[i]

        def choose_n(items,n):
            indexes = set()
            while True:
                indexes.add(rng.randint(0,len(items)-1))
                if len(indexes)==n: return [items[i] for i in indexes]

        for _ in range(sr):
            anchor, positive, negative = [], [], []
            for x,y in zip(self.train_X.tolist(),self.train_Y.tolist()):
                anchor.append(x)
                positive.append(choose_unique(by_class[y],x))
                negative.append(choose_n(by_class[1-y],neg))

            yield torch.tensor(anchor).float(), torch.tensor(positive).float(), torch.tensor(negative).float()

    def train(self):
        return self.train_X, self.train_Y

    def test(self):
        rng_indexes = cb.CobaRandom(self.params['rng']).shuffle(range(len(self.test_X)))
        return self.test_X[rng_indexes,:], self.test_Y[rng_indexes]

class MyEvaluator:
    def __init__(self, s1, ws):
        self.s1 = s1
        self.ws = ws
        self.params = {'s1': s1, 'ws':ws }

    def evaluate(self, env, lrn):        
        torch.set_num_threads(1)
        torch.manual_seed(1)
        
        X, Y = env.train()

        loss = torch.nn.L1Loss(reduction='none')
        torch_loader  = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X),batch_size=16,shuffle=True)

        f = FeedForward(self.s1)
        fopt = COCOB(f.parameters())
        f.train()

        for _ in range(self.ws):
            for (_X,) in torch_loader:
                fopt.zero_grad()
                loss(f(torch.nan_to_num(_X)),_X).nanmean().backward()
                fopt.step()

        f.eval()
        X, Y = env.test()
        for i in range(35):
            with torch.no_grad():
                yield {'l1':loss(f(torch.nan_to_num(X)),X).nanmean()}

def make_envs(X, Y, G):
    X, Y, G = torch.tensor(X).float(), torch.tensor(Y).float(), torch.tensor(G)
    all_equal = set()
    envs = []
    for rng in range(2):
        for g in sorted(set(G.tolist())):
            if (g==G).sum() < 50: continue
            trn_x,trn_y = X[g!=G],Y[g!=G]
            tst_x,tst_y = X[g==G],Y[g==G]

            if len(set(tst_y.tolist())) == 1:
                all_equal.add(g)
            else:
                envs.append(MyEnvironment(trn_x,trn_y,tst_x,tst_y,g,rng))
    if all_equal: print(f"All Equal, no environment added for {sorted(all_equal)}") 
    return envs

lrns = [ FeedForward(specs=[], rng=1) ]

vals = lambda x: [
    MyEvaluator((len(x),.2,45,'l','r',len(x)), 8),
    MyEvaluator((len(x),.2,45,'l','r',45,'l','r',len(x)), 8),
    MyEvaluator((len(x),.2,90,'l','r',len(x)), 8),
    MyEvaluator((len(x),.2,90,'l','r',90,'l','r',len(x)), 8),
]

envs = make_envs(X3,Y2,G)
cb.Experiment(envs,lrns,vals(X3[0])).run('../logs/outae11.log',processes=35,quiet=True)


All Equal, no environment added for [2, 25, 61, 82, 87, 89, 99, 103, 111, 117, 146, 196, 203, 227, 246, 311, 319, 357, 377, 390, 451, 455]
{'Learners': 1, 'Environments': 349, 'Interactions': 47565}


In [ ]:
import coba as cb

class FeedForward(torch.nn.Sequential):
    """A Generic implementation of Feedforward Neural Network"""

    class SkipModule(torch.nn.Module):
        def __init__(self, layers):
            super().__init__()
            self.layers = layers
        def forward(self,X):
            return X + self.layers(X)

    def make_layer(self,curr_dim,spec):
        if isinstance(spec,float):
            return torch.nn.Dropout(spec), curr_dim
        if curr_dim is None and isinstance(spec,int):
            return None, spec
        if isinstance(spec,int):
            return torch.nn.Linear(curr_dim,spec),spec
        if spec == 'r':
            return torch.nn.ReLU(),curr_dim
        if spec == 'l':
            return torch.nn.LayerNorm(curr_dim),curr_dim
        if spec == 'b':
            return torch.nn.BatchNorm1d(curr_dim), curr_dim
        if spec == 's':
            return torch.nn.Sigmoid(),curr_dim
        if isinstance(spec,list):                
            return FeedForward.SkipModule(FeedForward([curr_dim] + spec)), curr_dim
        raise Exception("Bad Layer")

    def __init__(self, specs, rng=1):
        """Instantiate a Feedfoward network according to specifications.

        Args:
            specs: A sequence of layer specifications as follows:
                -1 -- replaced with the input feature width
                <int> -- a LinearLayer with output width equal to <int>
                [0,1] -- a Dropout layer with the given probability
                'l' -- a LayerNorm
                'b' -- a BatchNorm1d
                'r' -- a ReLU layer
                's' -- a Sigmoid layer
                [] -- a skip layer with the given specifications
        """

        torch.manual_seed(rng)
        layers,width = [],None
        for spec in specs:
            layer,width = self.make_layer(width,spec)
            if layer: layers.append(layer)
        super().__init__(*(layers or [torch.nn.Identity()]))
        self.params = {"specs": specs, "rng": rng }

class MyEnvironment:
    def __init__(self, train_X, train_Y, test_X, test_Y, g, rng):
        self.params = {'pid': g, 'rng': rng}
        self.train_X = train_X
        self.train_Y = train_Y.float()
        self.test_X = test_X
        self.test_Y = test_Y.float()

    def ssl(self,neg,sr):

        by_class  = {y: self.train_X[(self.train_Y==y)].tolist() for y in set(self.train_Y.tolist())}
        rng = cb.CobaRandom(self.params['rng'])

        def choose_unique(items,item):
            while items[i:=rng.randint(0,len(items)-1)]==item:
                pass
            return items[i]

        def choose_n(items,n):
            indexes = set()
            while True:
                indexes.add(rng.randint(0,len(items)-1))
                if len(indexes)==n: return [items[i] for i in indexes]

        for _ in range(sr):
            anchor, positive, negative = [], [], []
            for x,y in zip(self.train_X.tolist(),self.train_Y.tolist()):
                anchor.append(x)
                positive.append(choose_unique(by_class[y],x))
                negative.append(choose_n(by_class[1-y],neg))

            yield torch.tensor(anchor).float(), torch.tensor(positive).float(), torch.tensor(negative).float()

    def train(self):
        return self.train_X, self.train_Y

    def test(self):
        rng_indexes = cb.CobaRandom(self.params['rng']).shuffle(range(len(self.test_X)))
        return self.test_X[rng_indexes,:], self.test_Y[rng_indexes]

class MyEvaluator:
    def __init__(self, s1, ws):
        self.s1 = s1
        self.ws = ws
        self.params = {'s1': s1, 'ws':ws }

    def evaluate(self, env, lrn):        
        torch.set_num_threads(1)
        torch.manual_seed(1)
        
        X, Y = env.train()

        loss = torch.nn.L1Loss(reduction='none')
        torch_loader  = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X),batch_size=16,shuffle=True)

        f = FeedForward(self.s1)
        fopt = COCOB(f.parameters())
        f.train()

        for _ in range(self.ws):
            for (_X,) in torch_loader:
                fopt.zero_grad()
                loss(f(torch.nan_to_num(_X)),_X).nanmean().backward()
                fopt.step()

        f.eval()
        X, Y = env.test()
        for i in range(35):
            with torch.no_grad():
                yield {'l1':loss(f(torch.nan_to_num(X)),X).nanmean()}

def make_envs(X, Y, G):
    X, Y, G = torch.tensor(X).float(), torch.tensor(Y).float(), torch.tensor(G)
    all_equal = set()
    envs = []
    for rng in range(2):
        for g in sorted(set(G.tolist())):
            if (g==G).sum() < 50: continue
            trn_x,trn_y = X[g!=G],Y[g!=G]
            tst_x,tst_y = X[g==G],Y[g==G]

            if len(set(tst_y.tolist())) == 1:
                all_equal.add(g)
            else:
                envs.append(MyEnvironment(trn_x,trn_y,tst_x,tst_y,g,rng))
    if all_equal: print(f"All Equal, no environment added for {sorted(all_equal)}") 
    return envs

lrns = [ FeedForward(specs=[], rng=1) ]

vals = lambda x: [
    MyEvaluator((len(x),90,'l','r',len(x)), 8),
    MyEvaluator((len(x),.2,90,'l','r',len(x)), 8),
    MyEvaluator((len(x),.2,90,'l','r',len(x)), 16),
]

envs = make_envs(X3,Y2,G)
cb.Experiment(envs,lrns,vals(X3[0])).run('../logs/outae12.log',processes=35,quiet=True)


All Equal, no environment added for [2, 25, 61, 82, 87, 89, 99, 103, 111, 117, 146, 196, 203, 227, 246, 311, 319, 357, 377, 390, 451, 455]
{'Learners': 1, 'Environments': 55, 'Interactions': 4515}
